## How To Bind User Parameters to a Quantum Kernel

In this guide, we show the ins and outs of binding user parameters to a quantum kernel in Qiskit. 

We can create a trainable `QuantumKernel` (`QK`) by specifying that some of our feature map's parameters are trainable parameters rather than inputs. This can be done at initialization by passing an array of `Parameters` as the `user_parameters` argument to the `QK` constructor. Alternatively, this can be done using the `QK.user_parameters` setter after initialization.

After the `QK.user_parameters` field has been set, `QK.assign_user_parameters()` offers two ways to bind user parameters

1. Bind user parameters using a dictionary
    - Keys to dict must be parameters within the feature map and must exist in `QK.user_parameters`
    - Values in dict may be either numerical assignments or `ParameterExpression` objects
2. Bind user parameters using a list of values
    - If binding using a list of values, the list must be of same size and ordering as `QK.user_parameters`. Each input value will be bound to its corresponding user_parameters value.
     
We begin by importing a few packages and instantiating a feature map circuit with three trainable user parameters, `θ`, and three input parameters, `x`.

In [1]:
#pylint: disable=import-error, wrong-import-position, pointless-statement
import os
import sys
import numpy as np

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_machine_learning.kernels import QuantumKernel

In [2]:
NUM_QUBITS = 3
fm = QuantumCircuit(NUM_QUBITS)
input_params = ip = ParameterVector('x', NUM_QUBITS)
user_params  = up = ParameterVector('θ', NUM_QUBITS)

for i in range(NUM_QUBITS):
    fm.h(i)
    fm.ry(up[i], i)

for i in range(NUM_QUBITS):
    fm.crx(ip[i], (i)%NUM_QUBITS,   (i+1)%NUM_QUBITS)

# Define a Quantum Kernel using our trainable feature map
qk = QuantumKernel(fm, user_parameters=user_params[:NUM_QUBITS])

print("input_params:", input_params)
print("user_params:", user_params)
qk.feature_map.draw()

input_params: x, ['x[0]', 'x[1]', 'x[2]']
user_params: θ, ['θ[0]', 'θ[1]', 'θ[2]']


┌───┐┌──────────┐                        ┌──────────┐
q_0: ┤ H ├┤ Ry(θ[0]) ├─────■──────────────────┤ Rx(x[2]) ├
     ├───┤├──────────┤┌────┴─────┐            └────┬─────┘
q_1: ┤ H ├┤ Ry(θ[1]) ├┤ Rx(x[0]) ├─────■───────────┼──────
     ├───┤├──────────┤└──────────┘┌────┴─────┐     │      
q_2: ┤ H ├┤ Ry(θ[2]) ├────────────┤ Rx(x[1]) ├─────■──────
     └───┘└──────────┘            └──────────┘

### Option  1: Bind User Parameters with a Dictionary

Here, we will use a dictionary of the form `{Parameter : Value}` that maps user parameters to either numeric values or `ParameterExpression` objects.

In [3]:
# Bind parameters to numeric values
param_binds = {up[0]: np.pi/2,
               up[1]: np.pi/3,
               up[2]: np.pi/4}

qk.assign_user_parameters(param_binds)
qk.feature_map.draw()

┌───┐┌─────────┐                        ┌──────────┐
q_0: ┤ H ├┤ Ry(π/2) ├─────■──────────────────┤ Rx(x[2]) ├
     ├───┤├─────────┤┌────┴─────┐            └────┬─────┘
q_1: ┤ H ├┤ Ry(π/3) ├┤ Rx(x[0]) ├─────■───────────┼──────
     ├───┤├─────────┤└──────────┘┌────┴─────┐     │      
q_2: ┤ H ├┤ Ry(π/4) ├────────────┤ Rx(x[1]) ├─────■──────
     └───┘└─────────┘            └──────────┘

We are free to bind a subset of our user parameters and re-bind parameters to new values.

In [4]:
# Create incomplete user param bindings
param_binds = {up[0]: np.pi/6,
               up[1]: np.pi/5}

qk.assign_user_parameters(param_binds)
qk.feature_map.draw()

┌───┐┌─────────┐                        ┌──────────┐
q_0: ┤ H ├┤ Ry(π/6) ├─────■──────────────────┤ Rx(x[2]) ├
     ├───┤├─────────┤┌────┴─────┐            └────┬─────┘
q_1: ┤ H ├┤ Ry(π/5) ├┤ Rx(x[0]) ├─────■───────────┼──────
     ├───┤├─────────┤└──────────┘┌────┴─────┐     │      
q_2: ┤ H ├┤ Ry(π/4) ├────────────┤ Rx(x[1]) ├─────■──────
     └───┘└─────────┘            └──────────┘

We can  un-bind our user-parameters or assign user parameters to different `ParameterExpression` objects. This is done in in the same way that we would bind numeric values. 

In [5]:
# Create incomplete user param bindings
param_binds = {up[0]: up[0],
               up[1]: up[0]+up[2],
               up[2]: up[2]}

qk.assign_user_parameters(param_binds)
qk.feature_map.draw()

┌───┐    ┌──────────┐                           ┌──────────┐
q_0: ┤ H ├────┤ Ry(θ[0]) ├────────■──────────────────┤ Rx(x[2]) ├
     ├───┤┌───┴──────────┴──┐┌────┴─────┐            └────┬─────┘
q_1: ┤ H ├┤ Ry(θ[0] + θ[2]) ├┤ Rx(x[0]) ├─────■───────────┼──────
     ├───┤└───┬──────────┬──┘└──────────┘┌────┴─────┐     │      
q_2: ┤ H ├────┤ Ry(θ[2]) ├───────────────┤ Rx(x[1]) ├─────■──────
     └───┘    └──────────┘               └──────────┘

### Option 2: Bind User Parameters with a List

If the `user_parameters` have been specified in the `QuantumKernel`, we may bind and unbind those parameters using only lists of parameter values. Note that the list of values must always be equal in size to the `QuantumKernel.user_parameters` array, and the values will be assigned in order.

Here we instantiate a new quantum kernel with the three user parameters unbound.

In [8]:
qk = QuantumKernel(fm, user_parameters=user_params)
qk.feature_map.draw()

┌───┐┌──────────┐                        ┌──────────┐
q_0: ┤ H ├┤ Ry(θ[0]) ├─────■──────────────────┤ Rx(x[2]) ├
     ├───┤├──────────┤┌────┴─────┐            └────┬─────┘
q_1: ┤ H ├┤ Ry(θ[1]) ├┤ Rx(x[0]) ├─────■───────────┼──────
     ├───┤├──────────┤└──────────┘┌────┴─────┐     │      
q_2: ┤ H ├┤ Ry(θ[2]) ├────────────┤ Rx(x[1]) ├─────■──────
     └───┘└──────────┘            └──────────┘

We may want to assign numerical values to parameters 0 and 2, while leaving parameter 1 unchanged.

In [10]:
param_values = [np.pi/7, up[1], np.pi/9]
qk.assign_user_parameters(param_values)
qk.feature_map.draw()

┌───┐┌─────────┐                         ┌──────────┐
q_0: ┤ H ├┤ Ry(π/7) ├──────■──────────────────┤ Rx(x[2]) ├
     ├───┤├─────────┴┐┌────┴─────┐            └────┬─────┘
q_1: ┤ H ├┤ Ry(θ[1]) ├┤ Rx(x[0]) ├─────■───────────┼──────
     ├───┤├─────────┬┘└──────────┘┌────┴─────┐     │      
q_2: ┤ H ├┤ Ry(π/9) ├─────────────┤ Rx(x[1]) ├─────■──────
     └───┘└─────────┘             └──────────┘

To assign parameter 1 to a numerical value, while leaving parameters 0 and 2 unchaged, we pass in a full list of the new values (values 0 and 2 will remain the same.)

In [11]:
param_values = [np.pi/7, np.pi/6, np.pi/9]
qk.assign_user_parameters(param_values)
qk.feature_map.draw()

┌───┐┌─────────┐                        ┌──────────┐
q_0: ┤ H ├┤ Ry(π/7) ├─────■──────────────────┤ Rx(x[2]) ├
     ├───┤├─────────┤┌────┴─────┐            └────┬─────┘
q_1: ┤ H ├┤ Ry(π/6) ├┤ Rx(x[0]) ├─────■───────────┼──────
     ├───┤├─────────┤└──────────┘┌────┴─────┐     │      
q_2: ┤ H ├┤ Ry(π/9) ├────────────┤ Rx(x[1]) ├─────■──────
     └───┘└─────────┘            └──────────┘

Finally, if we want to unbind all of our parameters, we may just pass in a list of the parameters themselves.

In [12]:
param_values = [up[0], up[1], up[2]]
qk.assign_user_parameters(param_values)
qk.feature_map.draw()

┌───┐┌──────────┐                        ┌──────────┐
q_0: ┤ H ├┤ Ry(θ[0]) ├─────■──────────────────┤ Rx(x[2]) ├
     ├───┤├──────────┤┌────┴─────┐            └────┬─────┘
q_1: ┤ H ├┤ Ry(θ[1]) ├┤ Rx(x[0]) ├─────■───────────┼──────
     ├───┤├──────────┤└──────────┘┌────┴─────┐     │      
q_2: ┤ H ├┤ Ry(θ[2]) ├────────────┤ Rx(x[1]) ├─────■──────
     └───┘└──────────┘            └──────────┘

In [1]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright